In [28]:
from dotenv import load_dotenv
import requests
import psycopg2
import os

load_dotenv()

True

In [29]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [30]:
from langchain.tools import tool
from tavily import TavilyClient

tavily = TavilyClient()
api_weather = os.getenv("WEATHER_API_KEY")

@tool
def get_capital(country: str) -> str:
    """Get the capital of a country."""
    capital = tavily.search(
        query=f"What is the capital of {country}?",
        search_depth="ultra-fast", # ultra-fast, fast, medium, advanced 
        max_results=1,
    )
    return capital

@tool(description="Get Realtime weather of a city.")
def get_weather(city: str) -> str:
    url = f"https://api.tomorrow.io/v4/weather/realtime?location={city}&apikey={api_weather}"
    headers = {
        "accept": "application/json",
        "accept-encoding": "deflate, gzip, br"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:  
        return f"Erro: status code {response.status_code}"

tools = [get_capital, get_weather] 

In [40]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_history(session_id: str):
    # String de conexão PostgreSQL para Supabase
    connection_string = "sqlite:///message_store.db"
    
    return SQLChatMessageHistory(
        session_id=session_id,
        connection=connection_string,
        table_name="message_store"
    )

In [41]:
from langchain_core.messages import trim_messages

trimmer = trim_messages(
    max_tokens=10,  # cada mensagem conta como 1 token
    strategy="last",   # seleciona as últimas mensagens
    token_counter=lambda x: 1,  # Cada mensagem conta como 1 token
)

In [47]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain_core.messages import HumanMessage
from rich.pretty import pprint


agent = create_agent( 
    model=model,
    tools=tools, 
    # response_format=ToolStrategy(AgentResponse),
)

'''
response = agent.invoke(
    {"messages": [HumanMessage(content="Ola me chamo patrick")]},
    config
)

pprint(response, expand_all=True) # melhora a visualização de dict/list no terminal
'''

'\nresponse = agent.invoke(\n    {"messages": [HumanMessage(content="Ola me chamo patrick")]},\n    config\n)\n\npprint(response, expand_all=True) # melhora a visualização de dict/list no terminal\n'

In [48]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente prestativo."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{user_input}")
])

In [49]:
trim_chain = {
    "user_input": lambda x: x["user_input"],
    "history": lambda x: trimmer.invoke(x["history"]),
}

chain = trim_chain | prompt | agent

In [50]:
from langchain_core.runnables.history import RunnableWithMessageHistory

runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="user_input",
    history_messages_key="history"
)

In [55]:
user_input = "qual o meu nome?"
response = runnable_with_history.invoke(
    {"user_input": user_input},
    config={"configurable": {"session_id": "1"}} 
)

In [56]:
pprint(response)

{
│   'messages': [
│   │   SystemMessage(
│   │   │   content='Você é um assistente prestativo.',
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='640a0a58-f04e-4d89-a570-468092e0249b'
│   │   ),
│   │   HumanMessage(
│   │   │   content='qual o meu nome?',
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='fce85601-20e9-4ca8-9df4-f5f2fa738f58'
│   │   ),
│   │   SystemMessage(
│   │   │   content='Você é um assistente prestativo.',
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='1b68f0e1-0915-4518-bc5b-e696fd3c858f'
│   │   ),
│   │   HumanMessage(
│   │   │   content='qual o meu nome?',
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='4e244a1c-620b-4e3f-a718-a0e85f293287'
│   │   ),
│   │   AIMessage(
│   │   │   content='Desculpe, mas eu não tenho essa informação. Como posso ajudar você hoje?',
│   │   │   additional_kwargs={'refusal': None},
│   │   │   response_metadata={
│   │   │   │   'token_usage': {
│   │   │   │   │   'completion_tokens': 22,
│   │   │   │   │   'prompt_tokens': 83,
│   │   │   │   │   'total_tokens': 105,
│   │   │   │   │   'completion_tokens_details': {
│   │   │   │   │   │   'accepted_prediction_tokens': 0,
│   │   │   │   │   │   'audio_tokens': 0,
│   │   │   │   │   │   'reasoning_tokens': 0,
│   │   │   │   │   │   'rejected_prediction_tokens': 0
│   │   │   │   │   },
│   │   │   │   │   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
│   │   │   │   },
│   │   │   │   'model_provider': 'openai',
│   │   │   │   'model_name': 'gpt-3.5-turbo-0125',
│   │   │   │   'system_fingerprint': None,
│   │   │   │   'id': 'chatcmpl-D07vPTva39UtVdNTANGXgINX0CwKs',
│   │   │   │   'service_tier': 'default',
│   │   │   │   'finish_reason': 'stop',
│   │   │   │   'logprobs': None
│   │   │   },
│   │   │   id='lc_run--019bdc04-66d7-75f0-808e-f8df7b8fab4d-0',
│   │   │   tool_calls=[],
│   │   │   invalid_tool_calls=[],
│   │   │   usage_metadata={
│   │   │   │   'input_tokens': 83,
│   │   │   │   'output_tokens': 22,
│   │   │   │   'total_tokens': 105,
│   │   │   │   'input_token_details': {'audio': 0, 'cache_read': 0},
│   │   │   │   'output_token_details': {'audio': 0, 'reasoning': 0}
│   │   │   }
│   │   ),
│   │   HumanMessage(
│   │   │   content='me chamo patrick, e voce?',
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='df5e549a-7ba5-43c1-91a2-d716bced68dd'
│   │   ),
│   │   SystemMessage(
│   │   │   content='Você é um assistente prestativo.',
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='d4a672a7-b25c-47f8-9e7c-32fdac7f95d6'
│   │   ),
│   │   HumanMessage(
│   │   │   content='qual o meu nome?',
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='80b24613-64d6-4d9f-a4bc-3289d2234238'
│   │   ),
│   │   HumanMessage(
│   │   │   content='me chamo patrick, e voce?',
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='e0643652-98d3-4362-abff-6bdb4b120764'
│   │   ),
│   │   AIMessage(
│   │   │   content='Olá, Patrick! Eu sou um assistente virtual aqui para te ajudar. Como posso ser útil hoje?',
│   │   │   additional_kwargs={'refusal': None},
│   │   │   response_metadata={
│   │   │   │   'token_usage': {
│   │   │   │   │   'completion_tokens': 26,
│   │   │   │   │   'prompt_tokens': 144,
│   │   │   │   │   'total_tokens': 170,
│   │   │   │   │   'completion_tokens_details': {
│   │   │   │   │   │   'accepted_prediction_tokens': 0,
│   │   │   │   │   │   'audio_tokens': 0,
│   │   │   │   │   │   'reasoning_tokens': 0,
│   │   │   │   │   │   'rejected_prediction_tokens': 0
│   │   │   │   │   },
│   │   │   │   │   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
│   │   │   │   },
│   │   │   │   'model_provider': 'openai',
│   │   │   │   'model_name': 'gpt-3.5-turbo-0125